# OpenAI Provider Guide 🚀

This notebook demonstrates how to use OpenAI's models through different hosting options:
- **Direct OpenAI**: Using OpenAI's native API
- **Azure OpenAI**: Using OpenAI models through Microsoft Azure

## Overview

OpenAI offers their models through multiple hosting platforms, each with distinct advantages:

### Direct OpenAI Hosting
- **Latest models** available immediately upon release
- **Global availability** with worldwide access
- **Simple authentication** with API key
- **Full feature set** including all model capabilities
- **Competitive pricing** with usage-based billing

### Azure OpenAI Hosting
- **Enterprise security** and compliance features
- **Regional deployment** for data residency requirements
- **VNet integration** for private network access
- **Microsoft ecosystem** integration
- **SLA guarantees** for production workloads
- **Content filtering** and safety features

## Setup

First, let's load environment variables for authentication:

In [5]:
from dotenv import load_dotenv
import os
import base64
import httpx

load_dotenv()

True

## Direct OpenAI Hosting

### Authentication
The OpenAI client automatically uses the `OPENAI_API_KEY` environment variable for authentication.

### Available Models (as of 2025)
- `gpt-4.1` - Latest and most capable model
- `gpt-4o` - Optimized for speed and efficiency
- `gpt-4-turbo` - Previous generation flagship model
- `gpt-3.5-turbo` - Fast and cost-effective option

### Key Features
- **Function calling** for tool integration
- **Structured outputs** with JSON mode
- **Vision capabilities** for image understanding
- **Code generation** and analysis

### Resources
- [Python SDK Documentation](https://platform.openai.com/docs/libraries?language=python)
- [API Reference](https://platform.openai.com/docs/api-reference)
- [Model Documentation](https://platform.openai.com/docs/models)

### Basic Message Example

**Note**: OpenAI recently updated their API with a new `responses` endpoint that provides enhanced capabilities:

In [2]:
from openai import OpenAI

# Initialize client (uses OPENAI_API_KEY from environment)
client = OpenAI()

# Using the new responses API for enhanced capabilities
response = client.responses.create(
    model="gpt-4.1",
    input="Write a creative one-sentence story about a time-traveling cat."
)

print("=== Direct OpenAI Response ===")
print(f"Model: {response.model}")
print(f"Status: {response.status}")

# Extract the message content
if response.output and len(response.output) > 0:
    message = response.output[0]
    print(f"Content: {message.content[0].text}")
    print(f"Role: {message.role}")

print(f"Usage: {response.usage.input_tokens} input + {response.usage.output_tokens} output = {response.usage.total_tokens} total tokens")

# Full response structure
print("\n=== Full Response Structure ===")
response.model_dump()

=== Direct OpenAI Response ===
Model: gpt-4.1-2025-04-14
Status: completed
Content: With a swish of her tail and a mischievous wink, Whiskers the tabby leapt through the laundry basket and landed in ancient Egypt, where she was immediately mistaken for a goddess and offered a golden pyramid of tuna.
Role: assistant
Usage: 22 input + 49 output = 71 total tokens

=== Full Response Structure ===


{'id': 'resp_683b61ac753081a0805500beedcd190f089e89ddbfb758aa',
 'created_at': 1748722092.0,
 'error': None,
 'incomplete_details': None,
 'instructions': None,
 'metadata': {},
 'model': 'gpt-4.1-2025-04-14',
 'object': 'response',
 'output': [{'id': 'msg_683b61ae2c4481a098c5d3a1e044a965089e89ddbfb758aa',
   'content': [{'annotations': [],
     'text': 'With a swish of her tail and a mischievous wink, Whiskers the tabby leapt through the laundry basket and landed in ancient Egypt, where she was immediately mistaken for a goddess and offered a golden pyramid of tuna.',
     'type': 'output_text'}],
   'role': 'assistant',
   'status': 'completed',
   'type': 'message'}],
 'parallel_tool_calls': True,
 'temperature': 1.0,
 'tool_choice': 'auto',
 'tools': [],
 'top_p': 1.0,
 'background': False,
 'max_output_tokens': None,
 'previous_response_id': None,
 'reasoning': {'effort': None, 'generate_summary': None, 'summary': None},
 'service_tier': 'default',
 'status': 'completed',
 'text':

### Chat Completions API

For traditional chat-style interactions, you can also use the chat completions endpoint:

In [4]:
# Traditional chat completions API
chat_response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful AI assistant specializing in creative writing."},
        {"role": "user", "content": "Write an opening line for a mystery novel."}
    ],
    max_tokens=100,
    temperature=0.7
)

print("=== Chat Completions Response ===")
print(f"Model: {chat_response.model}")
print(f"Content: {chat_response.choices[0].message.content}")
print(f"Finish reason: {chat_response.choices[0].finish_reason}")
print(f"Usage: {chat_response.usage.prompt_tokens} + {chat_response.usage.completion_tokens} = {chat_response.usage.total_tokens} tokens")

=== Chat Completions Response ===
Model: gpt-4o-2024-08-06
Content: The last thing Detective Marlowe expected to find in the abandoned lighthouse was a meticulously set dinner table for two.
Finish reason: stop
Usage: 31 + 23 = 54 tokens


## Function Calling (Tool Use)

OpenAI's function calling allows models to interact with external tools and APIs. Here's how to implement it:

### Key Features:
- **Parallel function calls**: Multiple tools can be called simultaneously
- **Structured schemas**: Tools use JSON schemas for parameter validation
- **Tool choice control**: Force specific tools or let the model choose
- **Conversation continuity**: Results can be fed back for extended interactions

### Single Function Call Example

In [6]:
# Define a weather function
weather_function = {
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country e.g. New York, USA"
                },
                "units": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "Temperature units"
                }
            },
            "required": ["location"],
            "additionalProperties": False
        }
    }
}

# Initial request with function
messages = [{"role": "user", "content": "What is the weather like in Paris today?"}]

function_response = client.chat.completions.create(
    model="gpt-4.1",
    messages=messages,
    tools=[weather_function],
    tool_choice="auto"  # Let the model decide when to use tools
)

print("=== Function Call Response ===")
print(f"Model: {function_response.model}")

message = function_response.choices[0].message
print(f"Content: {message.content}")

if message.tool_calls:
    print(f"\nFunction calls: {len(message.tool_calls)}")
    for i, tool_call in enumerate(message.tool_calls):
        print(f"  Call {i+1}:")
        print(f"    ID: {tool_call.id}")
        print(f"    Function: {tool_call.function.name}")
        print(f"    Arguments: {tool_call.function.arguments}")

# Add assistant's response to conversation
messages.append(message)

print(f"\nMessages so far: {len(messages)}")

=== Function Call Response ===
Model: gpt-4.1-2025-04-14
Content: None

Function calls: 1
  Call 1:
    ID: call_pOJkoy8lSK57rlYYod9WVlhk
    Function: get_weather
    Arguments: {"location":"Paris, France","units":"celsius"}

Messages so far: 2


### Function Execution and Response

In [7]:
import json

# Simulate function execution
def execute_weather_function(location, units="fahrenheit"):
    """Simulate getting weather data"""
    if "paris" in location.lower():
        if units == "celsius":
            return "20°C, sunny with light clouds"
        else:
            return "68°F, sunny with light clouds"
    else:
        return f"Weather data not available for {location}"

# Execute function calls if any
if message.tool_calls:
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_weather":
            # Parse function arguments
            args = json.loads(tool_call.function.arguments)
            location = args.get("location")
            units = args.get("units", "fahrenheit")
            
            # Execute the function
            weather_result = execute_weather_function(location, units)
            
            print(f"=== Function Execution ===")
            print(f"Function: {tool_call.function.name}")
            print(f"Location: {location}")
            print(f"Units: {units}")
            print(f"Result: {weather_result}")
            
            # Add function result to conversation
            messages.append({
                "role": "tool",
                "content": weather_result,
                "tool_call_id": tool_call.id
            })
    
    # Get final response with function results
    final_response = client.chat.completions.create(
        model="gpt-4.1",
        messages=messages,
        tools=[weather_function]
    )
    
    print(f"\n=== Final Response ===")
    print(f"GPT's response: {final_response.choices[0].message.content}")
else:
    print("No function calls found in response")

=== Function Execution ===
Function: get_weather
Location: Paris, France
Units: celsius
Result: 20°C, sunny with light clouds

=== Final Response ===
GPT's response: The weather in Paris today is 20°C, sunny with light clouds. It’s a pleasant day to be outside!


### Multiple Functions Example

In [8]:
# Define multiple functions
calculator_function = {
    "type": "function",
    "function": {
        "name": "calculator",
        "description": "Perform basic mathematical operations",
        "parameters": {
            "type": "object",
            "properties": {
                "operation": {
                    "type": "string",
                    "enum": ["add", "subtract", "multiply", "divide"],
                    "description": "The mathematical operation"
                },
                "a": {"type": "number", "description": "First number"},
                "b": {"type": "number", "description": "Second number"}
            },
            "required": ["operation", "a", "b"],
            "additionalProperties": False
        }
    }
}

search_function = {
    "type": "function",
    "function": {
        "name": "web_search",
        "description": "Search the web for information",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "Search query"
                },
                "max_results": {
                    "type": "integer",
                    "description": "Maximum number of results",
                    "default": 5
                }
            },
            "required": ["query"],
            "additionalProperties": False
        }
    }
}

# Request that might use multiple functions
multi_function_response = client.chat.completions.create(
    model="gpt-4.1",
    messages=[
        {
            "role": "user",
            "content": "Calculate 25 * 16, and also search for 'OpenAI GPT-4 capabilities'"
        }
    ],
    tools=[weather_function, calculator_function, search_function],
    parallel_tool_calls=True  # Enable parallel function calls
)

print("=== Multiple Functions Response ===")
print(f"Model: {multi_function_response.model}")

message = multi_function_response.choices[0].message
print(f"Content: {message.content}")

if message.tool_calls:
    print(f"\nFunction calls: {len(message.tool_calls)}")
    for i, tool_call in enumerate(message.tool_calls):
        print(f"  Call {i+1}:")
        print(f"    ID: {tool_call.id}")
        print(f"    Function: {tool_call.function.name}")
        print(f"    Arguments: {tool_call.function.arguments}")
        
print(f"\nUsage: {multi_function_response.usage.prompt_tokens} + {multi_function_response.usage.completion_tokens} = {multi_function_response.usage.total_tokens} tokens")

=== Multiple Functions Response ===
Model: gpt-4.1-2025-04-14
Content: None

Function calls: 2
  Call 1:
    ID: call_VKjGDrigCRuPxh8INMbFKBXW
    Function: calculator
    Arguments: {"operation": "multiply", "a": 25, "b": 16}
  Call 2:
    ID: call_bUwqQXSXQQFUu6k7qg0J31Rm
    Function: web_search
    Arguments: {"query": "OpenAI GPT-4 capabilities"}

Usage: 180 + 57 = 237 tokens


## Images and PDF-files

### Images
First let's try sending an image using a base64 encoded string (local file)

In [9]:
# For base64-encoded images
image1_url = "https://upload.wikimedia.org/wikipedia/commons/a/a7/Camponotus_flavomarginatus_ant.jpg"
image1_media_type = "image/jpeg"
image1_data = base64.standard_b64encode(httpx.get(image1_url).content).decode("utf-8")

response = client.responses.create(
    model="gpt-4.1",
    input=[
        {
            "role": "user",
            "content": [
                { "type": "input_text", "text": "what's in this image?" },
                {
                    "type": "input_image",
                    "image_url": f"data:image/jpeg;base64,{image1_data}",
                },
            ],
        }
    ],
)

print(response.output_text)

This is a close-up (macro) photograph of an ant. The image captures a side view of the ant, highlighting its segmented body, long legs, antennae, and detailed hairs on its body. The ant appears to be in a defensive or alert posture, possibly lifting its upper body and antennae. The background is blurred, focusing attention on the ant as the central subject.


You can also just pass a URL if it's a public image

In [8]:
response = client.responses.create(
    model="gpt-4.1-mini",
    input=[{
        "role": "user",
        "content": [
            {"type": "input_text", "text": "what's in this image?"},
            {
                "type": "input_image",
                "image_url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
            },
        ],
    }],
)

print(response.output_text)

This image shows a peaceful landscape with a wooden boardwalk stretching straight ahead through a grassy field. The sky is partly cloudy with a lot of blue visible, suggesting a pleasant day. On both sides of the boardwalk, there is tall green grass, and in the distance, there are some trees and shrubs. The scene evokes a sense of calm and natural beauty.


### PDF-Files
Use a base64 encoded PDF-file

In [18]:
# First, load and encode the PDF 
pdf_url = "https://assets.anthropic.com/m/1cd9d098ac3e6467/original/Claude-3-Model-Card-October-Addendum.pdf"
pdf_data = base64.standard_b64encode(httpx.get(pdf_url).content).decode("utf-8")

response = client.responses.create(
    model="gpt-4.1",
    input=[
        {
            "role": "user",
            "content": [
                {
                    "type": "input_file",
                    "filename": "Claude-3-Model-Card-October-Addendum.pdf",
                    "file_data": f"data:application/pdf;base64,{pdf_data}",
                },
                {
                    "type": "input_text",
                    "text": "Explain what this document is about in 2 sentences. ",
                },
            ],
        },
    ]
)

print(response.output_text)

This document is an addendum to the model card for Anthropic’s Claude 3.5 Sonnet and introduces both an upgraded version of Claude 3.5 Sonnet and the new Claude 3.5 Haiku, highlighting their enhanced capabilities in reasoning, coding, visual processing, and especially computer use (interacting with GUIs from screenshots). It provides detailed performance evaluations, safety considerations, benchmark results, and safety procedures to inform users and the AI community about these models' advancements and responsible deployment.


Directly pass a PDF-file to OpenAI

In [19]:
file = client.files.create(
    file=open("./assets/gameboy_color.pdf", "rb"),
    purpose="user_data"
)

file.model_dump()

{'id': 'file-Cnt6BvgXHajtnod2Da7Nsm',
 'bytes': 1042289,
 'created_at': 1748723552,
 'filename': 'gameboy_color.pdf',
 'object': 'file',
 'purpose': 'user_data',
 'status': 'processed',
 'expires_at': None,
 'status_details': None}

In [23]:
response = client.responses.create(
    model="gpt-4.1",
    input=[
        {
            "role": "user",
            "content": [
                {
                    "type": "input_file",
                    "file_id": file.id,
                },
                {
                    "type": "input_text",
                    "text": "Explain what this PDF is about in two sentences.",
                },
            ]
        }
    ]
)

print(response.output_text)

This PDF is the instruction booklet for the Nintendo Game Boy Color, providing setup, usage, troubleshooting information, and warranty details. It includes diagrams, safety instructions, and guidelines for playing and maintaining the Game Boy Color system.


In [22]:
response.model_dump()

{'id': 'resp_683b6770b5c8819c9f8ada170d6e976c0d259d9aa9e7bfa8',
 'created_at': 1748723571.0,
 'error': None,
 'incomplete_details': None,
 'instructions': None,
 'metadata': {},
 'model': 'gpt-4.1-2025-04-14',
 'object': 'response',
 'output': [{'id': 'msg_683b6774e298819cb00742da4f70e2900d259d9aa9e7bfa8',
   'content': [{'annotations': [],
     'text': 'This PDF is the instruction booklet for the Nintendo Game Boy Color handheld video game system. It provides information on setup, usage, troubleshooting, and warranty details for the device and its accessories.',
     'type': 'output_text'}],
   'role': 'assistant',
   'status': 'completed',
   'type': 'message'}],
 'parallel_tool_calls': True,
 'temperature': 1.0,
 'tool_choice': 'auto',
 'tools': [],
 'top_p': 1.0,
 'background': False,
 'max_output_tokens': None,
 'previous_response_id': None,
 'reasoning': {'effort': None, 'generate_summary': None, 'summary': None},
 'service_tier': 'default',
 'status': 'completed',
 'text': {'for

## Azure OpenAI

### Setup and Authentication
Azure OpenAI requires specific configuration including endpoint, API version, and deployment names.

### Key Differences:
- **Custom endpoint**: Your Azure OpenAI resource endpoint
- **API version**: Specific API version for compatibility
- **Deployment names**: Models are deployed with custom names
- **Regional availability**: Not all models available in all regions
- **Content filtering**: Additional safety and content filtering layers

### Available Regions:
- `eastus2` - United States East (primary for US models)
- `swedencentral` - Sweden Central (primary for EU models)
- `australiaeast` - Australia East
- See [Azure OpenAI Regions](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models) for complete list

### Important Notes:
- **Model deployment required**: You must deploy models through Azure AI Studio
- **Custom names**: Deployment names can be different from model names
- **Quota management**: Each region has specific quotas and limits

### Resources:
- [Azure OpenAI Documentation](https://learn.microsoft.com/en-us/azure/ai-services/openai/)
- [Azure AI Studio](https://ai.azure.com/)
- [Model Availability](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models#model-summary-table-and-region-availability)

### Basic Azure OpenAI Example

In [9]:
from openai import AzureOpenAI

# Initialize Azure OpenAI client
azure_client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2025-01-01-preview",  # Use latest API version
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

print(f"Azure OpenAI Configuration:")
print(f"  Endpoint: {os.getenv('AZURE_OPENAI_ENDPOINT')}")
print(f"  API Version: 2025-01-01-preview")

# Note: Model name should match your deployment name in Azure
azure_response = azure_client.chat.completions.create(
    model="gpt-4.1",  # This should match your deployment name
    messages=[
        {
            "role": "user",
            "content": "Explain the benefits of using Azure OpenAI over direct OpenAI.",
        },
    ],
    max_tokens=200,
    temperature=0.7
)

print("\n=== Azure OpenAI Response ===")
print(f"Model: {azure_response.model}")
print(f"Content: {azure_response.choices[0].message.content}")
print(f"Usage: {azure_response.usage.prompt_tokens} + {azure_response.usage.completion_tokens} = {azure_response.usage.total_tokens} tokens")

# System fingerprint for version tracking
if hasattr(azure_response, 'system_fingerprint'):
    print(f"System fingerprint: {azure_response.system_fingerprint}")

print("\n=== Full Response Structure ===")
azure_response.model_dump()

Azure OpenAI Configuration:
  Endpoint: https://vectrix-ai.openai.azure.com/
  API Version: 2025-01-01-preview

=== Azure OpenAI Response ===
Model: gpt-4.1-2025-04-14
Content: Certainly! **Azure OpenAI** and **OpenAI (direct)** both provide access to powerful language models like GPT-4, but using Azure OpenAI through Microsoft’s Azure cloud platform offers several distinct benefits, especially for enterprise and business use cases. Here’s a breakdown:

---

## **1. Enterprise-Grade Security & Compliance**

- **Data Privacy & Residency:** Azure allows you to keep your data within specific regions or countries, complying with local regulations (e.g., GDPR, HIPAA).
- **Advanced Security:** Integration with Azure security features like Azure Active Directory, role-based access control, private networking (VNET), and auditing.
- **Certifications:** Azure OpenAI is certified for many industry standards (ISO, SOC, etc.), which is often required for regulated industries.

---

## **2. Integra

{'id': 'chatcmpl-BctcDCmWidPuvkQRBh52t4Lu6PIRU',
 'choices': [{'finish_reason': 'length',
   'index': 0,
   'logprobs': None,
   'message': {'content': 'Certainly! **Azure OpenAI** and **OpenAI (direct)** both provide access to powerful language models like GPT-4, but using Azure OpenAI through Microsoft’s Azure cloud platform offers several distinct benefits, especially for enterprise and business use cases. Here’s a breakdown:\n\n---\n\n## **1. Enterprise-Grade Security & Compliance**\n\n- **Data Privacy & Residency:** Azure allows you to keep your data within specific regions or countries, complying with local regulations (e.g., GDPR, HIPAA).\n- **Advanced Security:** Integration with Azure security features like Azure Active Directory, role-based access control, private networking (VNET), and auditing.\n- **Certifications:** Azure OpenAI is certified for many industry standards (ISO, SOC, etc.), which is often required for regulated industries.\n\n---\n\n## **2. Integration with Az

### Azure OpenAI Function Calling

Function calling works identically on Azure OpenAI as with direct OpenAI:

In [10]:
# Function calling works the same on Azure OpenAI
azure_function_response = azure_client.chat.completions.create(
    model="gpt-4.1",  # Your Azure deployment name
    messages=[
        {"role": "user", "content": "Calculate 144 divided by 12"}
    ],
    tools=[calculator_function],  # Same function definition as before
    tool_choice="auto"
)

print("=== Azure OpenAI Function Call ===")
print(f"Model: {azure_function_response.model}")

message = azure_function_response.choices[0].message
print(f"Content: {message.content}")

if message.tool_calls:
    for tool_call in message.tool_calls:
        print(f"\nFunction call:")
        print(f"  Function: {tool_call.function.name}")
        print(f"  Arguments: {tool_call.function.arguments}")
        
        # Parse and display the calculation
        args = json.loads(tool_call.function.arguments)
        print(f"  Operation: {args['a']} {args['operation']} {args['b']}")

print(f"\n✅ Function calling works identically on Azure OpenAI!")

=== Azure OpenAI Function Call ===
Model: gpt-4.1-2025-04-14
Content: None

Function call:
  Function: calculator
  Arguments: {"operation":"divide","a":144,"b":12}
  Operation: 144 divide 12

✅ Function calling works identically on Azure OpenAI!


## Comparison: Direct OpenAI vs Azure OpenAI

Let's compare the same request across both hosting options:

In [11]:
test_messages = [
    {
        "role": "user",
        "content": "Explain machine learning in exactly 3 sentences."
    }
]

print("=== Comparison: Direct OpenAI vs Azure OpenAI ===")

# Direct OpenAI
direct_response = client.chat.completions.create(
    model="gpt-4o",
    messages=test_messages,
    max_tokens=150
)

print("\n🔵 Direct OpenAI:")
print(f"Model: {direct_response.model}")
print(f"Response: {direct_response.choices[0].message.content}")
print(f"Tokens: {direct_response.usage.prompt_tokens} + {direct_response.usage.completion_tokens}")

# Azure OpenAI
azure_response = azure_client.chat.completions.create(
    model="gpt-4.1",  # Your deployment name
    messages=test_messages,
    max_tokens=150
)

print("\n🟠 Azure OpenAI:")
print(f"Model: {azure_response.model}")
print(f"Response: {azure_response.choices[0].message.content}")
print(f"Tokens: {azure_response.usage.prompt_tokens} + {azure_response.usage.completion_tokens}")

print("\n💡 Both use identical APIs and response formats!")
print("💡 Key differences: authentication, endpoints, and enterprise features")

=== Comparison: Direct OpenAI vs Azure OpenAI ===

🔵 Direct OpenAI:
Model: gpt-4o-2024-08-06
Response: Machine learning is a branch of artificial intelligence that enables systems to learn and improve from experience without being explicitly programmed. It involves the use of algorithms and statistical models to analyze and draw inferences from patterns within data. This process allows machines to autonomously make predictions or decisions based on new, unseen data.
Tokens: 16 + 60

🟠 Azure OpenAI:
Model: gpt-4.1-2025-04-14
Response: Machine learning is a branch of artificial intelligence that enables computers to learn from data without being explicitly programmed. It involves creating algorithms that identify patterns and make predictions or decisions based on new data. This process improves over time as the system is exposed to more data and feedback.
Tokens: 16 + 54

💡 Both use identical APIs and response formats!
💡 Key differences: authentication, endpoints, and enterprise feature